In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

taxiDB = pd.read_csv('data/taxi_dataset.csv')

In [25]:
taxiDB.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N


<dl>
<dt> Описание колонок:
<dd>id - ID поездки </dd>
<dd>vendor_id - ID компании, осуществляющей перевозку </dd>
<dd>pickup_datetime - Таймкод начала поездки</dd>
<dd>dropoff_datetime - Таймкод конца поездки </dd>
<dd>passenger_count - Количество пассажиров </dd>
<dd>pickup_longitude - Долгота точки, в которой началась поездка </dd>
<dd>pickup_latitude - Широта точки, в которой началась поездка </dd>
<dd>dropoff_longitude - Долгота точки, в которой закончилась поездка </dd>
<dd>dropoff_latitude - Широта точки, в которой закончилась поездка </dd>
<dd>store_and_fwd_flag - Yes/No: Была ли информация сохранена в памяти транспортного средства из-за потери соединения с сервером </dd>
</dl>

**Целевая переменная - длительность поездки.**

Зная тайм-коды времени начала и конца поездок, можем вычислить обозначенный таргет.
Договоримся, что производим вычисления в секундах.

Положим таргетную переменнул в столбик с названием *trip_duration*

In [26]:
# приведем pickup_datetime и dropoff_datetime к типу datetime
taxiDB['pickup_datetime'] = pd.to_datetime(taxiDB['pickup_datetime'])
taxiDB['dropoff_datetime'] = pd.to_datetime(taxiDB['dropoff_datetime'])

In [27]:
# добавим столбец - таргет
taxiDB['trip_duration'] = (taxiDB['dropoff_datetime'] - taxiDB['pickup_datetime']).dt.total_seconds()

In [28]:
taxiDB.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455.0
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663.0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124.0
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429.0
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435.0


Предсказывая таргет для новых объектов в будущем, мы не будем заранее знать **dropoff_datetime**.

Удалим колонку из датасета.

In [29]:
taxiDB = taxiDB.drop('dropoff_datetime', axis=1)

Рассмотрим имеющиеся вещественные/бинарные признаки и решим, какие простейшие признаки можно вытащить из остальных колонок.

Во-первых, имеем бинарный признак `vendor_id`, принимающий значения {1, 2}. Переведем его во множество {0, 1}.

In [30]:
taxiDB['vendor_id'] = taxiDB['vendor_id'] - 1

Также закодируем бинарный признак `store_and_fwd_flag`. 'N' - 0, 'Y' - 1.

In [31]:
taxiDB['store_and_fwd_flag'] = taxiDB['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)

Во-вторых, можем использовать долготу и широту точек старта/завершения поездки, чтобы примерно оценить расстояние между 2 точками.

Базовая идея состоит в том, чтобы посчитать разность долгот и широт соответственно, то есть:

$$
\delta_{long} = \text{dropoff\_longitude} - \text{pickup\_longitude}
$$

$$
\delta_{lat} =  \text{dropoff\_latitude} - \text{pickup\_latitude}
$$

А потом вычислить географическое расстояние между 2 точками по теореме Пифагора:

$$
R = \sqrt{\delta^2_{long} + \delta^2_{lat}}
$$

Мы реализуем данную задумку и вычислим вещественную колонку *R*.

Только для начала нужно некоторым образом перевести долготу и широту в километры, обеспечив равенство их мер измерения. Потому что, вообще говоря, **градусная мера** широт и долгот имеет неодинаковую шкалу перевода в километры. 

Начнем переводить каждую долготу в некоторое относительно километровое выражение. Соберем список из всех широт и посчитаем медиану

In [32]:
medianLat = pd.concat([taxiDB['pickup_latitude'], taxiDB['dropoff_latitude']]).median()
medianLat

40.75431823730469

Теперь, из каждого значения широты вычтем медианное значение.

Результат переведем в километры.

In [33]:
latMultiplier  = 111.32

taxiDB['pickup_latitude']   = latMultiplier  * (taxiDB['pickup_latitude']   - medianLat)
taxiDB['dropoff_latitude']   = latMultiplier  * (taxiDB['dropoff_latitude']  - medianLat)

Итого, для **latitude** колонок получили следующие выражения:

*На сколько примерно километров севернее или южнее (в зависимости от знака) точка находится относительно средней широты*

In [34]:
medianLong  = pd.concat([taxiDB['pickup_longitude'], taxiDB['dropoff_longitude']]).median()

longMultiplier = np.cos(medianLat*(np.pi/180.0)) * 111.32

In [35]:
taxiDB['pickup_longitude']   = longMultiplier  * (taxiDB['pickup_longitude']   - medianLong)
taxiDB['dropoff_longitude']   = longMultiplier  * (taxiDB['dropoff_longitude']  - medianLong)

In [36]:
taxiDB.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,1,2016-03-14 17:24:55,1,-0.110015,1.516008,1.367786,1.256121,0,455.0
1,id2377394,0,2016-06-12 00:43:35,1,0.036672,-1.753813,-1.571088,-2.578912,0,663.0
2,id3858529,1,2016-01-19 11:35:24,1,0.153763,1.070973,-2.064547,-4.923841,0,2124.0
3,id3504673,1,2016-04-06 19:32:31,1,-2.461500,-3.823568,-2.649362,-5.298809,0,429.0
4,id2181028,1,2016-03-26 13:30:55,1,0.657515,4.329328,0.668452,3.139453,0,435.0


Наконец, вычислим географическое расстояние **distance_km**:

In [37]:
taxiDB['distance_km'] = ((taxiDB['dropoff_latitude'] - taxiDB['pickup_latitude']) ** 2 
+ (taxiDB['dropoff_longitude'] - taxiDB['pickup_longitude']) ** 2) ** (1/2)

In [38]:
taxiDB.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance_km
0,id2875421,1,2016-03-14 17:24:55,1,-0.110015,1.516008,1.367786,1.256121,0,455.0,1.500479
1,id2377394,0,2016-06-12 00:43:35,1,0.036672,-1.753813,-1.571088,-2.578912,0,663.0,1.807119
2,id3858529,1,2016-01-19 11:35:24,1,0.153763,1.070973,-2.064547,-4.923841,0,2124.0,6.392080
3,id3504673,1,2016-04-06 19:32:31,1,-2.461500,-3.823568,-2.649362,-5.298809,0,429.0,1.487155
4,id2181028,1,2016-03-26 13:30:55,1,0.657515,4.329328,0.668452,3.139453,0,435.0,1.189925


Уберем старые признаки

In [39]:
taxiDB = taxiDB.drop(['pickup_longitude', 'dropoff_longitude',
                      'pickup_latitude', 'dropoff_latitude', 'pickup_datetime'], axis=1)

In [40]:
taxiDB.head()

,id,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
0,id2875421,1,1,0,455.0,1.500479
1,id2377394,0,1,0,663.0,1.807119
2,id3858529,1,1,0,2124.0,6.392080
3,id3504673,1,1,0,429.0,1.487155
4,id2181028,1,1,0,435.0,1.189925


Посмотрим, какие значения может принимать `passenger_count`.

In [41]:
taxiDB['passenger_count'].value_counts()

passenger_count
1    1033540
2     210318
5      78088
3      59896
6      48333
4      28404
0         60
7          3
9          1
8          1
Name: count, dtype: int64

Количество пассажиров от поездки к поездке ограничено. Вряд ли если к нам придут новые данные, мы увидим числа бОльшие, чем у нас в датасете. Тогда предположим, что данный признак является категориальным

Заменим колонку **passenger_count** колонкой **category_encoded**, используя  **Mean-target encoding**

In [42]:
taxiDB['passenger_count'] = taxiDB['passenger_count'].map(taxiDB.groupby('passenger_count')['trip_duration'].mean()) 
taxiDB.head()

,id,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
0,id2875421,1,930.399753,0,455.0,1.500479
1,id2377394,0,930.399753,0,663.0,1.807119
2,id3858529,1,930.399753,0,2124.0,6.392080
3,id3504673,1,930.399753,0,429.0,1.487155
4,id2181028,1,930.399753,0,435.0,1.189925


Остались две колонки: **id**, **pickup_datetime**

**id** можно использовать как обычный идентификатор нашего объекта, поэтому поместите данную колонку в качестве индекса нашей таблички:

In [43]:
taxiDB = taxiDB.set_index('id')

In [44]:
taxiDB.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,
id2875421,1,930.399753,0,455.0,1.500479
id2377394,0,930.399753,0,663.0,1.807119
id3858529,1,930.399753,0,2124.0,6.392080
id3504673,1,930.399753,0,429.0,1.487155
id2181028,1,930.399753,0,435.0,1.189925


Сохраним подготовленные данные

In [45]:
taxiDB.to_csv('output/processed_data.csv', sep=';')